# data

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd


path = "/content/drive/My Drive/lecture/l_d/p9/mard/"
data = pd.read_csv(path + "user_song_overall.csv", encoding="utf-8")

In [2]:
data.head()

,Unnamed: 0,reviewerID,songs_x,overall
0,0,A1OFY4ATO7D13W,Southtown Siren,5.0
1,1,A2KH83L1F70QR8,Southtown Siren,5.0
2,2,A1OFY4ATO7D13W,Don't Know Me At All,5.0
3,3,A2KH83L1F70QR8,Don't Know Me At All,5.0
4,4,A1OFY4ATO7D13W,Texas Sky,5.0


In [0]:
data = data.iloc[:250000, 1:]

In [0]:
data.columns = ["userID", "itemID", "rating"]

In [5]:
df = data
df.head()

,userID,itemID,rating
0,A1OFY4ATO7D13W,Southtown Siren,5.0
1,A2KH83L1F70QR8,Southtown Siren,5.0
2,A1OFY4ATO7D13W,Don't Know Me At All,5.0
3,A2KH83L1F70QR8,Don't Know Me At All,5.0
4,A1OFY4ATO7D13W,Texas Sky,5.0


In [6]:
pd.unique(df.iloc[:, 0]).size

23697

In [7]:
pd.unique(df.iloc[:, 1]).size

52570

In [8]:
!pip install surprise

In [0]:
from surprise import Reader
from surprise import dataset


reader = Reader(line_format='user item rating', rating_scale=(1, 5))

# Also, a dummy Dataset class
class MyDataset(dataset.DatasetAutoFolds):

    def __init__(self, df, reader):

        self.raw_ratings = [(uid, iid, r, None) for (uid, iid, r) in
                            zip(df['userID'], df['itemID'], df['rating'])]
        self.reader=reader

data = MyDataset(df, reader)


In [21]:
data.raw_ratings

[('A1OFY4ATO7D13W', 'Southtown Siren', 5.0, None),
 ('A2KH83L1F70QR8', 'Southtown Siren', 5.0, None),
 ('A1OFY4ATO7D13W', "Don't Know Me At All", 5.0, None),
 ('A2KH83L1F70QR8', "Don't Know Me At All", 5.0, None),
 ('A1OFY4ATO7D13W', 'Texas Sky', 5.0, None),
 ('A2KH83L1F70QR8', 'Texas Sky', 5.0, None),
 ('A1OFY4ATO7D13W', 'Love Stories', 5.0, None),
 ('A2KH83L1F70QR8', 'Love Stories', 5.0, None),
 ('A1OFY4ATO7D13W', 'Southern Fashion', 5.0, None),
 ('A2KH83L1F70QR8', 'Southern Fashion', 5.0, None),
 ('A1OFY4ATO7D13W', 'Parallel Line', 5.0, None),
 ('A2KH83L1F70QR8', 'Parallel Line', 5.0, None),
 ('A1OFY4ATO7D13W', 'Liquor & Guilt', 5.0, None),
 ('A2KH83L1F70QR8', 'Liquor & Guilt', 5.0, None),
 ('A1OFY4ATO7D13W', "Ain't My Kind", 5.0, None),
 ('A2KH83L1F70QR8', "Ain't My Kind", 5.0, None),
 ('A1OFY4ATO7D13W', 'Casinos & Strip Shows', 5.0, None),
 ('A2KH83L1F70QR8', 'Casinos & Strip Shows', 5.0, None),
 ('A10W40DOIE1EOP', 'Become the Media', 2.0, None),
 ('A2D04UKNBWTSA3', 'Jok Omako Nya

# bsl

In [0]:
from surprise.model_selection import KFold
import numpy as np
import surprise


bsl_options = {
    'method': 'als',
    'n_epochs': 5,
#    'reg_u': 12, #defualt 10
#    'reg_i': 5   #defualt 15
}
algo = surprise.BaselineOnly(bsl_options)
def do(algo):

    

    np.random.seed(0)
    acc = np.zeros(3)
    cv = KFold(3)
    for i, (trainset, testset) in enumerate(cv.split(data)):
        algo.fit(trainset)
        predictions = algo.test(testset)
        acc[i] = surprise.accuracy.rmse(predictions, verbose=True)
    acc.mean()

In [26]:
do(algo)

Estimating biases using als...
RMSE: 0.6534
Estimating biases using als...
RMSE: 0.6588
Estimating biases using als...
RMSE: 0.6570


In [11]:
from surprise.model_selection import cross_validate

cross_validate(algo, data)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'fit_time': (0.1613321304321289,
  0.16736125946044922,
  0.17223906517028809,
  0.15693163871765137,
  0.16344451904296875),
 'test_mae': array([0.43087344, 0.43075275, 0.43328352, 0.43185407, 0.43688876]),
 'test_rmse': array([0.59071712, 0.59367591, 0.59859312, 0.59248558, 0.60246102]),
 'test_time': (0.2339484691619873,
  0.12395143508911133,
  0.23328757286071777,
  0.12376666069030762,
  0.12245035171508789)}

# mse 유사도

In [11]:
sim_options = {'name': 'msd'}
algo = surprise.KNNBasic(sim_options=sim_options)
do(algo)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8045
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7948
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.8019


# cosine 유사도

In [13]:
sim_options = {'name': 'cosine'}
algo = surprise.KNNBasic(sim_options=sim_options)
do(algo)

Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0171
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0109
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0161


# Pearson 유사도


In [14]:
sim_options = {'name': 'pearson'}
algo = surprise.KNNBasic(sim_options=sim_options)
do(algo)

Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9553
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9517
Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.9534


# Pearson-Baseline 유사도

In [15]:
sim_options = {'name': 'pearson_baseline'}
algo = surprise.KNNBasic(sim_options=sim_options)
do(algo)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.7293
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.7222
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.7259


# KNN 가중치 예측

In [16]:
sim_options = {'name': 'pearson_baseline'}
algo = surprise.KNNWithMeans(sim_options=sim_options)
do(algo)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5274
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5319
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5360


In [17]:
sim_options = {'name': 'pearson_baseline'}
algo = surprise.KNNBaseline(sim_options=sim_options)
do(algo)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5464
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5428
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
RMSE: 0.5436


# SVD




In [18]:
%%time
algo = surprise.SVD(n_factors=100)
do(algo)

RMSE: 0.5624
RMSE: 0.5608
RMSE: 0.5602
CPU times: user 42.1 s, sys: 319 ms, total: 42.4 s
Wall time: 42.5 s


# NMF 

In [19]:
%%time
algo = surprise.NMF(n_factors=100)
do(algo)

RMSE: 0.6845
RMSE: 0.6877
RMSE: 0.6896
CPU times: user 3min 24s, sys: 1.65 s, total: 3min 25s
Wall time: 3min 25s
